# 자연어 감성분석
- 감성사전 기반: 미리정의된 감성 단어 사전 사용(규칙 사용)
  - TextBlob, AFINNm VADER
- 머신러닝 기반: 데이터로부터 패턴 학습(통계 기반)
  - TF-IDF 벡터화
  - 선형회귀
  - 로지스틱회귀
  - F1 score, Recision, Recall -> classification report
- 사용데이터:
  - NLTK 영화 리뷰(2000개)
  - 다음 영화리뷰

---


# 1. 감정분석: 감성사전 기반

## 알고리즘
- TextBlob: 사전기반 감성분석
- AFINN: 감정 점수 매핑
- VADER(Valance Aware Dictionary): 소셜미디어 최적화 감성분석
- TF-IDF: 텍스트 벡터화
- Multinomial Navie Bayes: 확률 기반 분류
- Liner Regression
- Logistic Regression

### 1.1. TextBlob
```
이 영화는 정말 좋고 재미있다
- 좋다 +1(긍정)
- 재미있다 +1(긍정)
- +2 > 0 --> 긍정(pos) 분류
```
**Polarity(극성도)**
- (긍정단어 개수 - 부정단어 개수) / 전체단어 개수
- 범위: -1.0~+1.0  (0:중립)

**Subjectivity(주관성)**
- 평가대상 단어의 비율
- 범위: 0.0~1.0 (0:객관적, 1:주관적)

문맥을 무시하고 단어의 극성만 고려함.
```
이 영화는 나쁘지 않다. -> 나쁘다(-) *않다(-)*로 인식함
장점: 빠른 속도, 학습이 불필요
사용: 실시간 감성분석, 스트리밍 데이터
```


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('brown')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
from textblob import TextBlob, Word

text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob = TextBlob(text)

print(blob.sentences)
print('-'*200)
print(blob.words)
print('-'*200)
print(blob.tags)
print('-'*200)
print(blob.noun_phrases)

[Sentence("TextBlob is amazingly simple to use."), Sentence("What a wonderful library for NLP!")]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
['TextBlob', 'is', 'amazingly', 'simple', 'to', 'use', 'What', 'a', 'wonderful', 'library', 'for', 'NLP']
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[('TextBlob', 'NNP'), ('is', 'VBZ'), ('amazingly', 'RB'), ('simple', 'JJ'), ('to', 'TO'), ('use', 'VB'), ('What', 'WP'), ('a', 'DT'), ('wonderful', 'JJ'), ('library', 'NN'), ('for', 'IN'), ('NLP', 'NNP')]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 감성분석
print(blob.sentiment)
print('-'*100)
print(f'Polarity: {blob.sentiment.polarity}')
print(f'Subjectivity: {blob.sentiment.subjectivity}')

Sentiment(polarity=0.5, subjectivity=0.6785714285714286)
----------------------------------------------------------------------------------------------------
Polarity: 0.5
Subjectivity: 0.6785714285714286


### 1.2. AFINN
- Lexicon-Based
- 각 단어의 `-5~+5`의 점수를 부여하고 합산
```
이 영화는 좋지만, 좋지 않은 부분도 있다.
- 좋다+3 좋다+3 나쁘다-3  =>  +3>0 (긍정)
```
- `score = sum(word_sentiment_value)`
- 분류규칙: `score>0`(긍정), `score<0`(부정)
- 이모티콘 지원
- 강도표현 인식 `very, really..`
- 강조 수정자(Intensifiers)
  - 매우 좋다 = 1.5*(좋다의 점수)

---

TextBlob vs. AFINN
- TextBlob: 더 일반적인 접근
- AFINN: 더 정확한 점수 매핑

In [ ]:
from afinn import Afinn

af = Afinn()

text1 = "TextBlob is amazingly simple to use."
text2 = "What a wonderful library for NLP!"

score1 = af.score(text1)
score2 = af.score(text2)

score1, score2

(0.0, 4.0)

### 1.3. VADER
- rule-based
- **소셜미디어 텍스트**에 최적화
```
이 영화는 정말정말 훌륭해!!!
- 훌룽하다(기본) +0.7
- 정말정말(강조) x1.5
- !!!(문장부호 강조) x1.2
```
- **4개의 감정지수**
  - positive (긍정확률) `0~1`
  - negative (부정확률) `0~1`
  - neutral (중립확률) `0~1`
  - compound (종합점수) `-1~1`

- **Score**
```
score = compound_score / sqrt(compound_score**2 + 0.0625)
- score >= 0.05 (긍정)
- score <= -0.05 (부정)
- -0.05 < score < 0.05  (중립)
```

- **대소문자 구분**
```
AMAZING amazing 다른 점수
```
- **이모티콘 지원** -> 소셜미디어 text 최적화
```
:) 긍정 :-( 부정
```


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

sentences = [
    "I love this product! It's absolutely amazing 😍",
    "This is the worst movie I've ever seen...",
    "The food was okay, not great but not bad either.",
    "I’m REALLY happy with the results!!!",
    "Not good at all. I’m disappointed.",
]

In [ ]:
for s in sentences:
  scores = analyzer.polarity_scores(s)
  print(f'Sentence -> {s}')
  print(f'Score -> {scores}')
  print('-'*100)

Sentence -> I love this product! It's absolutely amazing 😍
Score -> {'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.862}
----------------------------------------------------------------------------------------------------
Sentence -> This is the worst movie I've ever seen...
Score -> {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}
----------------------------------------------------------------------------------------------------
Sentence -> The food was okay, not great but not bad either.
Score -> {'neg': 0.149, 'neu': 0.487, 'pos': 0.364, 'compound': 0.4728}
----------------------------------------------------------------------------------------------------
Sentence -> I’m REALLY happy with the results!!!
Score -> {'neg': 0.0, 'neu': 0.472, 'pos': 0.528, 'compound': 0.7651}
----------------------------------------------------------------------------------------------------
Sentence -> Not good at all. I’m disappointed.
Score -> {'neg': 0.579, 'neu': 0.421, 'pos':

In [ ]:
import nltk
from nltk.corpus import movie_reviews
from textblob import TextBlob
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# nltk 데이터 다운로드
nltk.download('movie_reviews', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# 영화리뷰 데이터 로드
fileids = movie_reviews.fileids()

In [ ]:
reviews = [movie_reviews.raw(fileid) for fileid in fileids[:50]] + [movie_reviews.raw(fileid) for fileid in fileids[-50:]]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids[:50]] + [movie_reviews.categories(fileid)[0] for fileid in fileids[-50:]]

len(reviews), categories.count('pos'), categories.count('neg')

(100, 50, 50)

In [ ]:
from sklearn.metrics import accuracy_score
# TextBlob
def sentiment_textblob(docs):
  return ['pos' if TextBlob(doc).sentiment.polarity > 0 else 'neg' for doc in docs]

predictions_textblob = sentiment_textblob(reviews)
accuracy_textblob = accuracy_score(categories, predictions_textblob)
print(f'TextBlob Accuracy: {accuracy_textblob}')


TextBlob Accuracy: 0.61


In [ ]:
# AFINN
def sentiment_afinn(docs):
  afn = Afinn()
  return ['pos' if afn.score(doc) > 0 else 'neg' for doc in docs]

prediction_afinn = sentiment_afinn(reviews)
accuracy_afinn = accuracy_score(categories, prediction_afinn)
print(f'AFINN Accuracy: {accuracy_afinn}')

AFINN Accuracy: 0.72


In [ ]:
# VADER
def sentiment_vader(docs):
  analyzer = SentimentIntensityAnalyzer()
  return ['pos' if analyzer.polarity_scores(doc)['compound'] > 0 else 'neg' for doc in docs]

prediction_vader = sentiment_vader(reviews)
accuracy_vader = accuracy_score(categories, prediction_vader)
print(f'VADER Accuracy: {accuracy_vader}')

VADER Accuracy: 0.64


# 2. 감성분석: 머신러닝 기반

- 나이브 베이즈
  - 베이즈 정리에 기반한 확률적 분류 모델
  ```
  "좋다"단어를 본 후 이 리뷰가 긍정일 확률은?
  - p(긍정|"좋다") = (p("좋다"|긍정) x p(긍정)) / p("좋다")
  ```
- **문서**나 **리뷰** 같은 텍스트 데이터에서, <br>
**"이 문장이 긍정일 확률 vs 부정일 확률 중 어느 쪽이 더 큰가?"** <br>
를 계산해 감정을 분류

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [64]:
# 데이터 분
dataset = train_test_split(reviews, categories, test_size=0.2, random_state=42, stratify=categories)

len(dataset[0]), len(dataset[2])

(80, 80)

In [65]:
# tf-idf 벡터화
vectorizer = TfidfVectorizer(max_features=1000)

X_train = vectorizer.fit_transform(dataset[0])
X_test = vectorizer.transform(dataset[1])
y_train = dataset[2]
y_test = dataset[3]

In [66]:
# 1. mnb
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train, y_train)
mnb_pred = mnb_clf.predict(X_test)

print(classification_report(y_test, mnb_pred))

              precision    recall  f1-score   support

         neg       0.54      0.70      0.61        10
         pos       0.57      0.40      0.47        10

    accuracy                           0.55        20
   macro avg       0.55      0.55      0.54        20
weighted avg       0.55      0.55      0.54        20



In [67]:
# 2. LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

         neg       0.62      0.80      0.70        10
         pos       0.71      0.50      0.59        10

    accuracy                           0.65        20
   macro avg       0.66      0.65      0.64        20
weighted avg       0.66      0.65      0.64        20



# 3. 성능향상
- 소문자 변환
- 연속된 문자열 중, 3글자 이상
- 어간추출(형태소 분석)
- 불용어 제거

In [68]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [73]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [74]:
def custom_tokenizer(text):
    text = text.lower()
    tokenizer =  RegexpTokenizer(r"[\w']{3,}")
    tokens = tokenizer.tokenize(text)
    porter = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    return [porter.stem(token) for token in tokens if token not in stop_words]
vector = TfidfVectorizer(
    tokenizer  = custom_tokenizer
    ,max_features=1000
    ,min_df=5
    ,max_df=0.5
    ,token_pattern = r"[\w']{3,}"
)
x_train = vector.fit_transform(dataset[0])
x_test = vector.transform(dataset[1])

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [75]:
def evaluate_model(model):
    model.fit(x_train,y_train)
    predict = model.predict(x_test)
    print( classification_report(y_test, predict))

In [76]:
evaluate_model(LogisticRegression())

              precision    recall  f1-score   support

         neg       0.80      0.80      0.80        10
         pos       0.80      0.80      0.80        10

    accuracy                           0.80        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.80      0.80      0.80        20

